In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable as V
import torch.nn.functional as F
from  sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import pandas as pd
from torch.utils.data import DataLoader,Dataset,TensorDataset
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
import numpy
from sklearn.model_selection import train_test_split
from collections import OrderedDict

In [66]:
# load in datasets
scnv_512=pd.read_csv("/Users/vivit/Capstone/data_scnv 512.csv")
scnv_256=pd.read_csv("/Users/vivit/Capstone/data_scnv 256.csv")
scnv_128=pd.read_csv("/Users/vivit/Capstone/data_scnv 128.csv")
train_clinical=pd.read_csv("/Users/vivit/Capstone/train_clinical_B.csv")
test_clinical=pd.read_csv("/Users/vivit/Capstone/test_clinical_B.csv")

In [67]:
train_idx=train_clinical[['attrib_name']]
test_idx=test_clinical[['attrib_name']]

In [68]:
scnv_512

,Unnamed: 0,1q21.3,1q44,3p25.1,3q26.32,4q13.3,5p15.33,6p23,6q21,8p11.23,...,0.433,0.434,0.435,0.436,0.437,0.438,0.439,0.440,0.441,0.442
0,TCGA.3C.AAAU,3.6569,0.1874,-0.4576,0.2586,3.6569,0.1659,0.1470,-1.0883,0.1976,...,0,0,0,0,0,0,0,0,0,0
1,TCGA.3C.AALI,0.3873,0.3858,0.4438,0.4765,-0.4365,0.0226,-0.0087,-0.4901,0.4473,...,0,0,0,0,0,0,0,0,0,0
2,TCGA.3C.AALJ,0.0833,0.4121,0.0000,-0.1448,-0.3167,0.0127,1.1985,0.0375,3.6569,...,0,0,0,0,0,0,0,0,0,0
3,TCGA.3C.AALK,0.5887,0.6172,-0.0162,-0.0577,-0.0326,0.0231,0.0134,0.0134,0.0264,...,0,0,0,0,0,0,0,0,0,0
4,TCGA.4H.AAAK,0.0000,0.0000,0.0046,-0.0028,-0.0039,0.0338,-0.0193,-0.0102,0.0171,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,TCGA.WT.AB44,1.8555,1.8555,-0.0856,-0.0856,-0.3846,0.0591,0.0146,0.0146,-0.1805,...,0,0,0,0,0,0,0,0,0,0
612,TCGA.XX.A899,0.2754,0.2754,0.0011,0.0011,-0.0003,0.3554,0.0039,0.0039,2.2140,...,0,0,0,0,0,0,0,0,0,0
613,TCGA.XX.A89A,0.0122,0.0043,-0.2560,-0.2653,-0.0028,0.0085,-0.0004,0.0000,-0.5301,...,0,0,0,0,0,0,0,0,0,0
614,TCGA.Z7.A8R5,0.4855,0.4964,-0.1585,-0.1585,0.1377,0.1467,0.1528,0.1372,0.0849,...,0,0,0,0,0,0,0,0,0,0


In [69]:
trainset___512 = scnv_512.merge(train_idx,left_on='Unnamed: 0', right_on='attrib_name',how="inner")
testset___512 = scnv_512.merge(test_idx,left_on='Unnamed: 0', right_on='attrib_name',how="inner")
trainset___256 = scnv_256.merge(train_idx,left_on='Unnamed: 0', right_on='attrib_name',how="inner")
testset___256 = scnv_256.merge(test_idx,left_on='Unnamed: 0', right_on='attrib_name',how="inner")
trainset___128 = scnv_128.merge(train_idx,left_on='Unnamed: 0', right_on='attrib_name',how="inner")
testset___128 = scnv_128.merge(test_idx,left_on='Unnamed: 0', right_on='attrib_name',how="inner")

In [71]:
trainset__512=trainset___512.drop("Unnamed: 0",axis=1)
testset__512=testset___512.drop("Unnamed: 0",axis=1)
trainset__256=trainset___256.drop("Unnamed: 0",axis=1)
testset__256=testset___256.drop("Unnamed: 0",axis=1)
trainset__128=trainset___128.drop("Unnamed: 0",axis=1)
testset__128=testset___128.drop("Unnamed: 0",axis=1)

In [72]:
trainset__512

,1q21.3,1q44,3p25.1,3q26.32,4q13.3,5p15.33,6p23,6q21,8p11.23,8p11.21,...,0.434,0.435,0.436,0.437,0.438,0.439,0.440,0.441,0.442,attrib_name
0,3.6569,0.1874,-0.4576,0.2586,3.6569,0.1659,0.1470,-1.0883,0.1976,0.1976,...,0,0,0,0,0,0,0,0,0,TCGA.3C.AAAU
1,0.3873,0.3858,0.4438,0.4765,-0.4365,0.0226,-0.0087,-0.4901,0.4473,0.4473,...,0,0,0,0,0,0,0,0,0,TCGA.3C.AALI
2,0.0833,0.4121,0.0000,-0.1448,-0.3167,0.0127,1.1985,0.0375,3.6569,-0.7089,...,0,0,0,0,0,0,0,0,0,TCGA.3C.AALJ
3,0.5887,0.6172,-0.0162,-0.0577,-0.0326,0.0231,0.0134,0.0134,0.0264,0.0264,...,0,0,0,0,0,0,0,0,0,TCGA.3C.AALK
4,0.0000,0.0000,0.0046,-0.0028,-0.0039,0.0338,-0.0193,-0.0102,0.0171,0.0171,...,0,0,0,0,0,0,0,0,0,TCGA.4H.AAAK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,0.9387,0.9387,0.0104,0.5114,-0.0297,0.5012,0.0083,0.0083,3.6569,3.3558,...,0,0,0,0,0,0,0,0,0,TCGA.WT.AB41
427,0.2754,0.2754,0.0011,0.0011,-0.0003,0.3554,0.0039,0.0039,2.2140,2.7946,...,0,0,0,0,0,0,0,0,0,TCGA.XX.A899
428,0.0122,0.0043,-0.2560,-0.2653,-0.0028,0.0085,-0.0004,0.0000,-0.5301,0.0118,...,0,0,0,0,0,0,0,0,0,TCGA.XX.A89A
429,0.4855,0.4964,-0.1585,-0.1585,0.1377,0.1467,0.1528,0.1372,0.0849,0.0849,...,0,0,0,0,0,0,0,0,0,TCGA.Z7.A8R5


In [73]:
trainset_512=trainset__512.drop("attrib_name",axis=1)
testset_512=testset__512.drop("attrib_name",axis=1)
trainset_256=trainset__256.drop("attrib_name",axis=1)
testset_256=testset__256.drop("attrib_name",axis=1)
trainset_128=trainset__128.drop("attrib_name",axis=1)
testset_128=testset__128.drop("attrib_name",axis=1)

In [74]:
trainset_512

,1q21.3,1q44,3p25.1,3q26.32,4q13.3,5p15.33,6p23,6q21,8p11.23,8p11.21,...,0.433,0.434,0.435,0.436,0.437,0.438,0.439,0.440,0.441,0.442
0,3.6569,0.1874,-0.4576,0.2586,3.6569,0.1659,0.1470,-1.0883,0.1976,0.1976,...,0,0,0,0,0,0,0,0,0,0
1,0.3873,0.3858,0.4438,0.4765,-0.4365,0.0226,-0.0087,-0.4901,0.4473,0.4473,...,0,0,0,0,0,0,0,0,0,0
2,0.0833,0.4121,0.0000,-0.1448,-0.3167,0.0127,1.1985,0.0375,3.6569,-0.7089,...,0,0,0,0,0,0,0,0,0,0
3,0.5887,0.6172,-0.0162,-0.0577,-0.0326,0.0231,0.0134,0.0134,0.0264,0.0264,...,0,0,0,0,0,0,0,0,0,0
4,0.0000,0.0000,0.0046,-0.0028,-0.0039,0.0338,-0.0193,-0.0102,0.0171,0.0171,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,0.9387,0.9387,0.0104,0.5114,-0.0297,0.5012,0.0083,0.0083,3.6569,3.3558,...,0,0,0,0,0,0,0,0,0,0
427,0.2754,0.2754,0.0011,0.0011,-0.0003,0.3554,0.0039,0.0039,2.2140,2.7946,...,0,0,0,0,0,0,0,0,0,0
428,0.0122,0.0043,-0.2560,-0.2653,-0.0028,0.0085,-0.0004,0.0000,-0.5301,0.0118,...,0,0,0,0,0,0,0,0,0,0
429,0.4855,0.4964,-0.1585,-0.1585,0.1377,0.1467,0.1528,0.1372,0.0849,0.0849,...,0,0,0,0,0,0,0,0,0,0


In [75]:
trainset512=trainset_512.to_numpy()
testset512=testset_512.to_numpy()
trainset256=trainset_256.to_numpy()
testset256=testset_256.to_numpy()
trainset128=trainset_128.to_numpy()
testset128=testset_128.to_numpy()

In [76]:
# standardize inputs
MMScaler=MinMaxScaler()
train_scaler512=MMScaler.fit_transform(trainset512)
test_scaler512=MMScaler.fit_transform(testset512)
train_scaler256=MMScaler.fit_transform(trainset256)
test_scaler256=MMScaler.fit_transform(testset256)
train_scaler128=MMScaler.fit_transform(trainset128)
test_scaler128=MMScaler.fit_transform(testset128)

In [77]:
train_torch512 = torch.FloatTensor(train_scaler512)
train_loader512 = torch.utils.data.DataLoader(train_torch512, batch_size = 16, shuffle=True)
test_torch512 = torch.FloatTensor(test_scaler512)
test_loader512 = torch.utils.data.DataLoader(test_torch512, batch_size = 16, shuffle=False)

train_torch256 = torch.FloatTensor(train_scaler256)
train_loader256 = torch.utils.data.DataLoader(train_torch256, batch_size = 16, shuffle=True)
test_torch256 = torch.FloatTensor(test_scaler256)
test_loader256 = torch.utils.data.DataLoader(test_torch256, batch_size = 16, shuffle=False)

train_torch128 = torch.FloatTensor(train_scaler128)
train_loader128 = torch.utils.data.DataLoader(train_torch128, batch_size = 16, shuffle=True)
test_torch128 = torch.FloatTensor(test_scaler128)
test_loader128 = torch.utils.data.DataLoader(test_torch128, batch_size = 16, shuffle=False)

In [78]:
# define parameters
EPOCH=10
lr=0.001

In [79]:
train_torch512.shape

torch.Size([431, 512])

In [80]:
test_torch512.shape

torch.Size([185, 512])

In [38]:
class Autoencoder512_2(nn.Module):
    def __init__(self):
        super(Autoencoder512_2, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128)           
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=512)
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return decoded
    
    
model512_2 = Autoencoder512_2()
if torch.cuda.is_available():
    model512_2.cuda()
    
model512_2.train(mode=True)
# define optimizer
optimizer512_2 = torch.optim.Adam(model512_2.parameters(), lr=lr)
criterion512_2=nn.MSELoss()


In [39]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch512:
        output = model512_2(batch_features)
        loss = criterion512_2(output, batch_features)
        optimizer512_2.zero_grad()
        loss.backward()
        optimizer512_2.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0038
epoch [2/10], loss:0.0033
epoch [3/10], loss:0.0031
epoch [4/10], loss:0.0027
epoch [5/10], loss:0.0023
epoch [6/10], loss:0.0020
epoch [7/10], loss:0.0021
epoch [8/10], loss:0.0018
epoch [9/10], loss:0.0017
epoch [10/10], loss:0.0018


In [40]:
class Autoencoder512_1(nn.Module):
    def __init__(self):
        super(Autoencoder512_1, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU()
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=256, out_features=512),
            nn.ReLU()
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return decoded
    
    
model512_1 = Autoencoder512_1()
if torch.cuda.is_available():
    model512_1.cuda()
    
model512_1.train(mode=True)
# define optimizer
optimizer512_1 = torch.optim.Adam(model512_1.parameters(), lr=lr)
criterion512_1=nn.MSELoss()

In [41]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch512:
        output = model512_1(batch_features)
        loss = criterion512_1(output, batch_features)
        optimizer512_1.zero_grad()
        loss.backward()
        optimizer512_1.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0080
epoch [2/10], loss:0.0043
epoch [3/10], loss:0.0032
epoch [4/10], loss:0.0021
epoch [5/10], loss:0.0017
epoch [6/10], loss:0.0015
epoch [7/10], loss:0.0013
epoch [8/10], loss:0.0012
epoch [9/10], loss:0.0010
epoch [10/10], loss:0.0010


In [59]:
feature=model512_1.encoder(train_torch512)
feature

tensor([[0.1365, 0.0000, 0.0000,  ..., 0.0000, 0.4500, 0.3724],
        [0.2410, 0.0000, 0.0000,  ..., 0.0000, 0.0692, 0.0869],
        [0.2014, 0.0000, 0.0000,  ..., 0.0000, 0.2171, 0.1193],
        ...,
        [0.0729, 0.0000, 0.0000,  ..., 0.0000, 0.0266, 0.2010],
        [0.1026, 0.0000, 0.0000,  ..., 0.0000, 0.1745, 0.1237],
        [0.2765, 0.0000, 0.0000,  ..., 0.0000, 0.1297, 0.0946]],
       grad_fn=<ReluBackward0>)

In [60]:
a = feature.detach().numpy()
print(a)

[[0.1364743  0.         0.         ... 0.         0.44997185 0.37239167]
 [0.24103168 0.         0.         ... 0.         0.0692184  0.08694164]
 [0.2014033  0.         0.         ... 0.         0.21711068 0.11934201]
 ...
 [0.07286714 0.         0.         ... 0.         0.02659992 0.20096286]
 [0.10255396 0.         0.         ... 0.         0.17446691 0.12372899]
 [0.27646714 0.         0.         ... 0.         0.12971444 0.09459502]]


In [61]:
a.shape

(431, 256)

In [62]:
np.savetxt("/Users/vivit/Capstone/SCNV AE results for Breast Cancer 73.csv",a)

In [64]:
torch.save(model512_1, '/Users/vivit/Capstone/AE model of SCNV for Breast cancer 73.pkl')

In [42]:
class Autoencoder256(nn.Module):
    def __init__(self):
        super(Autoencoder256, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU()
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU()
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return encoded,decoded
    
    
model256 = Autoencoder256()
if torch.cuda.is_available():
    model256.cuda()
    
model256.train(mode=True)
# define optimizer
optimizer256 = torch.optim.Adam(model256.parameters(), lr=lr)
criterion256=nn.MSELoss()

In [43]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch256:
        _,output = model256(batch_features)
        loss = criterion256(output, batch_features)    
        optimizer256.zero_grad()
        loss.backward()
        optimizer256.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0141
epoch [2/10], loss:0.0077
epoch [3/10], loss:0.0057
epoch [4/10], loss:0.0050
epoch [5/10], loss:0.0045
epoch [6/10], loss:0.0028
epoch [7/10], loss:0.0025
epoch [8/10], loss:0.0023
epoch [9/10], loss:0.0022
epoch [10/10], loss:0.0021


In [53]:
from sklearn.model_selection import KFold,StratifiedKFold
KF=KFold(n_splits=10,shuffle=True,random_state=123)
print(KF)

KFold(n_splits=10, random_state=123, shuffle=True)


In [54]:
def train_epoch(model,device,dataloader,loss_fun,optimizer):
    train_loss=0.0
    model.train()
    for x in dataloader:
        x=x.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fun(output,x)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss=train_loss/len(dataloader)
    return train_loss
def valid_epoch(model,device,dataloader,loss_fun):
    valid_loss = 0.0
    model.eval()
    for x in dataloader:
        x=x.to(device)
        output = model(x)
        loss=loss_func(output,x)
        valid_loss+=loss.item()
    valid_loss+=valid_loss/len(dataloader)
    return valid_loss

In [81]:
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset 
criterion=nn.MSELoss()
K_valid_loss=[]
for fold, (train_idx,val_idx) in enumerate(KF.split(np.arange(len(test_torch512)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = torch.utils.data.DataLoader(train_torch512,batch_size=512,sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(test_torch512,batch_size=512,sampler=test_sampler)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Autoencoder512_1()
    model.to(device)
    loss_func=nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    history = {'train_loss': [], 'test_loss': []}
    for epoch in range(EPOCH):
        train_loss=train_epoch(model,device,train_loader,criterion,optimizer)
        test_loss=valid_epoch(model,device,test_loader,criterion)
        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} ".format(epoch + 1,EPOCH,train_loss, test_loss))

Fold 1
Epoch:1/10 AVG Training Loss:0.017 AVG Test Loss:0.040 
Epoch:2/10 AVG Training Loss:0.016 AVG Test Loss:0.038 
Epoch:3/10 AVG Training Loss:0.014 AVG Test Loss:0.035 
Epoch:4/10 AVG Training Loss:0.013 AVG Test Loss:0.032 
Epoch:5/10 AVG Training Loss:0.012 AVG Test Loss:0.029 
Epoch:6/10 AVG Training Loss:0.011 AVG Test Loss:0.025 
Epoch:7/10 AVG Training Loss:0.010 AVG Test Loss:0.023 
Epoch:8/10 AVG Training Loss:0.008 AVG Test Loss:0.020 
Epoch:9/10 AVG Training Loss:0.008 AVG Test Loss:0.019 
Epoch:10/10 AVG Training Loss:0.007 AVG Test Loss:0.018 
Fold 2
Epoch:1/10 AVG Training Loss:0.017 AVG Test Loss:0.039 
Epoch:2/10 AVG Training Loss:0.016 AVG Test Loss:0.036 
Epoch:3/10 AVG Training Loss:0.014 AVG Test Loss:0.033 
Epoch:4/10 AVG Training Loss:0.013 AVG Test Loss:0.029 
Epoch:5/10 AVG Training Loss:0.012 AVG Test Loss:0.026 
Epoch:6/10 AVG Training Loss:0.010 AVG Test Loss:0.022 
Epoch:7/10 AVG Training Loss:0.009 AVG Test Loss:0.019 
Epoch:8/10 AVG Training Loss:0.00